In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np 
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from collections import Counter
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
train_dir = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/train'
val_dir = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/val'
test_dir = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/test'

In [ ]:
fig, ax = plt.subplots(5,5, figsize=(18,18))

for class_id in range(5):
    folder = os.path.join(train_dir,str(class_id))
    os.chdir(folder)
    samples = random.sample(os.listdir(folder), 5)
    
    for col in range(5):
        image = cv2.imread(samples[col])
        ax[class_id, col].imshow(image)
        ax[class_id, col].set_title("class_" + str(class_id))
        ax[class_id, col].set_axis_off()
    
plt.show()

In [ ]:
image_counts = {}
total_images = 0
for class_dir in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, class_dir)):
        class_count = len(os.listdir(os.path.join(train_dir, class_dir)))
        total_images += class_count
        image_counts[class_dir] = class_count

for class_id, count in image_counts.items():
    percentage = round(count / total_images * 100, 2)
    print('Class {}: {} images ({:.2f}%)'.format(class_id, count, percentage))

In [ ]:
NUM_CLASSES = 5
IMAGE_SIZE=[224, 224]
BATCH_SIZE=32

In [ ]:
import os
import shutil

# set the paths of the train and validation datasets
train_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/train'
val_path = '/kaggle/input/knee-osteoarthritis-dataset-with-severity/val'

# set the output directory path
output_dir = '/kaggle/working/output'
os.makedirs(output_dir, exist_ok=True)

# process the input files and save the output files to the output directory
for i in range(5):
    train_subfolder_path = os.path.join(train_path, str(i))
    val_subfolder_path = os.path.join(val_path, str(i))
    output_subfolder_path = os.path.join(output_dir, str(i))
    os.makedirs(output_subfolder_path, exist_ok=True)
    
    # process the train files and save the output files to the output subfolder
    if os.path.exists(train_subfolder_path):
        files = os.listdir(train_subfolder_path)
        for f in files:
            # process the file and save the output file to the output subfolder
            output_path = os.path.join(output_subfolder_path, f)
            shutil.copy(os.path.join(train_subfolder_path, f), output_path)

    # process the validation files and save the output files to the output subfolder
    if os.path.exists(val_subfolder_path):
        files = os.listdir(val_subfolder_path)
        for f in files:
            # process the file and save the output file to the output subfolder
            output_path = os.path.join(output_subfolder_path, f)
            shutil.copy(os.path.join(val_subfolder_path, f), output_path)

In [ ]:
train_dir="/kaggle/working/output"
image_counts = {}
total_images = 0
for class_dir in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, class_dir)):
        class_count = len(os.listdir(os.path.join(train_dir, class_dir)))
        total_images += class_count
        image_counts[class_dir] = class_count

for class_id, count in image_counts.items():
    percentage = round(count / total_images * 100, 2)
    print('Class {}: {} images ({:.2f}%)'.format(class_id, count, percentage))

In [ ]:
# Define a list of directories where your images are located
data_dirs = ['/kaggle/working/output/3', '/kaggle/working/output/4']

# Define the number of augmented images you want to create for each original image
augmentation_factor = 4

# Define the image generator for data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

# Define a loop to iterate through all the directories
for data_dir in data_dirs:
    # Define a loop to iterate through all the images in the directory
    for filename in os.listdir(data_dir):
        if filename.endswith('.png'): # check if it is an image file
            # Load the image and convert it to a Numpy array
            img = load_img(os.path.join(data_dir, filename))
            x = img_to_array(img)
            x = x.reshape((1,) + x.shape) # reshape to match the expected input shape of the generator
            
            # Generate augmented images and save them to the same directory
            i = 0
            for batch in datagen.flow(x, batch_size=1, save_to_dir=data_dir, save_prefix=filename.split('.')[0], save_format='jpg'):
                i += 1
                if i >= augmentation_factor:
                    break


In [ ]:
image_counts = {}
total_images = 0
for class_dir in os.listdir(train_dir):
    if os.path.isdir(os.path.join(train_dir, class_dir)):
        class_count = len(os.listdir(os.path.join(train_dir, class_dir)))
        total_images += class_count
        image_counts[class_dir] = class_count

for class_id, count in image_counts.items():
    percentage = round(count / total_images * 100, 2)
    print('Class {}: {} images ({:.2f}%)'.format(class_id, count, percentage))

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Load the MobileNetV2 model pre-trained on ImageNet without the top classification layer
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the first 144 layers of the model
for layer in base_model.layers[:144]:
    layer.trainable = False

# Add a new classification layer on top of the pre-trained model
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)



# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up data generator for training with data augmentation
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)
train_generator = train_datagen.flow_from_directory(
    directory='/kaggle/working/output',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Train the model for 30 epochs with data augmentation
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=30,
    verbose=1
)

In [ ]:
# specify the custom location to save the model
model_path = '/kaggle/working/my_model.h5'

# save the model in the H5 format to the custom location
model.save(model_path)

In [ ]:
# Set up data generator for test data
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    directory='/kaggle/input/knee-osteoarthritis-dataset-with-severity/test',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input

# Load the MobileNetV2 model pre-trained on ImageNet without the top classification layer
base_model = MobileNetV2(include_top=False, weights='imagenet', input_shape=(224, 224, 3))

# Freeze the first 144 layers of the model
for layer in base_model.layers[:144]:
    layer.trainable = False

# Add a new classification layer on top of the pre-trained model
x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.models.Model(inputs=base_model.input, outputs=x)

# Early stopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


train_generator = train_datagen.flow_from_directory(
    directory='/kaggle/working/output',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=True
)

# Train the model for 30 epochs with data augmentation
model.fit(
    train_generator,
    steps_per_epoch=train_generator.n // train_generator.batch_size,
    epochs=30,
    verbose=1,
    callbacks=[early_stop]
)


In [ ]:
# specify the custom location to save the model
model_path = '/kaggle/working/my_modelv2.h5'

# save the model in the H5 format to the custom location
model.save(model_path)

In [ ]:
# Set up data generator for test data
test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory(
    directory='/kaggle/input/knee-osteoarthritis-dataset-with-severity/test',
    target_size=(224, 224),
    color_mode='rgb',
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Evaluate the model on the test data
loss, accuracy = model.evaluate(test_generator, verbose=1)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

